In [139]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(20)

polars.config.Config

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [140]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [141]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())
print(data.collect().limit(3))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None
shape: (3, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬──────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---          │
│ str    ┆ str                        ┆ f64       ┆ f64        ┆ str          │
╞════════╪════════════════════════════╪═══════════╪════════════╪══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 28.617308 ┆ -88.647827 ┆ engine_check │
│ car_0  ┆ 2023-01-01T00:11:21.137484 ┆ 28.617308 ┆ -88.647827 ┆ null         │
│ car_0  ┆ 2023-01-01T00:00:20.878089 ┆ 28.617308 ┆ -88.647827 ┆ null         │
└────────┴────────────────────────────┴───────────┴────────────┴──────────────┘


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [142]:
# Convertir la columna `timestamp` usando múltiples formatos
# Todo tiene que ser convertido primero al mismo formato para que pueda ser parseado, no pueden haber 2 formatos de date distintos
data = data.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)
data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))
print(data.collect())

/tmp/ipykernel_3556/1201754905.py:9: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))


shape: (6_293_072, 5)
┌─────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id  ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---     ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str     ┆ datetime[μs]               ┆ f64       ┆ f64        ┆ str           │
╞═════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0   ┆ 2023-01-01 00:00:00        ┆ 28.617308 ┆ -88.647827 ┆ engine_check  │
│ car_0   ┆ 2023-01-01 00:11:21.000137 ┆ 28.617308 ┆ -88.647827 ┆ null          │
│ car_0   ┆ 2023-01-01 00:00:20.000878 ┆ 28.617308 ┆ -88.647827 ┆ null          │
│ car_0   ┆ 2023-01-01 00:05:29.000851 ┆ 28.617308 ┆ -88.647827 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01 00:23:17.000592 ┆ 28.617308 ┆ -88.647827 ┆ tire_pressure │
│ car_0   ┆ 2023-01-01 00:17:26.000368 ┆ 28.617308 ┆ -88.647827 ┆ low_fuel      │
│ car_0   ┆ 2023-01-01 00:02:25.000918 ┆ 28.617308 ┆ -88.647827 ┆ engine_che

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [ ]:
# Agrega la nueva columna `notification_time`
data_diff = data.clone()
data = data.sort(["car_id","timestamp"])
data_diff = (
    data_diff
    .sort(["car_id", "timestamp"])  # Ordenar por vehículo y tiempo
    .group_by("car_id", maintain_order=True)  # Agrupar por vehículo
    .agg([
        pl.col("timestamp").diff().alias("notification_time")  # Calcular diferencia de tiempo
    ])
    .explode("notification_time")
    .drop("car_id")
)
data = pl.concat([data, data_diff], how="horizontal")
print(data.collect())

shape: (6_293_072, 6)
┌─────────┬─────────────────────┬───────────┬────────────┬───────────────┬───────────────────┐
│ car_id  ┆ timestamp           ┆ latitude  ┆ longitude  ┆ notification  ┆ notification_time │
│ ---     ┆ ---                 ┆ ---       ┆ ---        ┆ ---           ┆ ---               │
│ str     ┆ datetime[μs]        ┆ f64       ┆ f64        ┆ str           ┆ duration[μs]      │
╞═════════╪═════════════════════╪═══════════╪════════════╪═══════════════╪═══════════════════╡
│ car_0   ┆ 2023-01-01 00:00:00 ┆ 28.617308 ┆ -88.647827 ┆ engine_check  ┆ null              │
│ car_0   ┆ 2023-01-01          ┆ 28.617308 ┆ -88.647827 ┆ null          ┆ 20s 878µs         │
│         ┆ 00:00:20.000878     ┆           ┆            ┆               ┆                   │
│ car_0   ┆ 2023-01-01          ┆ 28.617308 ┆ -88.647827 ┆ engine_check  ┆ 2m 5s 40µs        │
│         ┆ 00:02:25.000918     ┆           ┆            ┆               ┆                   │
│ car_0   ┆ 2023-01-01      

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [144]:
import numpy as np

# Función para calcular la distancia usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])  # Convertir a radianes
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c  # Distancia en kilómetros

# Calcular la distancia y agregarla como una nueva columna
data = (
    data
    .sort(["car_id", "timestamp"])  # Ordenar por vehículo y tiempo
    .with_columns(
        pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),  # Latitud previa
        pl.col("longitude").shift(1).over("car_id").alias("prev_longitude")  # Longitud previa
    )
    .with_columns(
        pl.struct(["latitude", "longitude", "prev_latitude", "prev_longitude"]).map_elements(
            lambda row: haversine(
                row["latitude"], row["longitude"], row["prev_latitude"], row["prev_longitude"]
            ) if row["prev_latitude"] is not None else None
        , return_dtype=pl.Float64).alias("distance")
    )
    .drop(["prev_latitude", "prev_longitude"])
)

print(data.collect().limit(5))


shape: (5, 7)
┌────────┬───────────────────┬───────────┬────────────┬──────────────┬──────────────────┬──────────┐
│ car_id ┆ timestamp         ┆ latitude  ┆ longitude  ┆ notification ┆ notification_tim ┆ distance │
│ ---    ┆ ---               ┆ ---       ┆ ---        ┆ ---          ┆ e                ┆ ---      │
│ str    ┆ datetime[μs]      ┆ f64       ┆ f64        ┆ str          ┆ ---              ┆ f64      │
│        ┆                   ┆           ┆            ┆              ┆ duration[μs]     ┆          │
╞════════╪═══════════════════╪═══════════╪════════════╪══════════════╪══════════════════╪══════════╡
│ car_0  ┆ 2023-01-01        ┆ 28.617308 ┆ -88.647827 ┆ engine_check ┆ null             ┆ null     │
│        ┆ 00:00:00          ┆           ┆            ┆              ┆                  ┆          │
│ car_0  ┆ 2023-01-01        ┆ 28.617308 ┆ -88.647827 ┆ null         ┆ 20s 878µs        ┆ 0.0      │
│        ┆ 00:00:20.000878   ┆           ┆            ┆              ┆       

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [ ]:
# Parámetro para definir el tamaño de los intervalos (en minutos)
x = 55
# Calcular estadísticas en intervalos dinámicos
df_intervals = data.clone()
df_intervals = (
    df_intervals
    .sort(["car_id", "timestamp"])  # Ordenar por vehículo y tiempo
    .group_by_dynamic(
        index_column="timestamp",  # Columna de tiempo para dividir en intervalos
        every=f"{x}m",             # Tamaño de los intervalos
        by="car_id",               # Agrupar dinámicamente por vehículo
        include_boundaries=True    # Incluir información sobre los límites del intervalo
    )
    .agg([
        # Estadísticas para `notification_time`
        pl.col("notification_time").mean().alias("notification_time_mean"),
        pl.col("notification_time").median().alias("notification_time_median"),
        pl.col("notification_time").var().alias("notification_time_variance"),
        pl.col("notification_time").max().alias("notification_time_max"),
        pl.col("notification_time").min().alias("notification_time_min"),
        # Estadísticas para `distance`
        pl.col("distance").mean().alias("distance_mean"),
        pl.col("distance").median().alias("distance_median"),
        pl.col("distance").var().alias("distance_variance"),
        pl.col("distance").max().alias("distance_max"),
        pl.col("distance").min().alias("distance_min"),
    ])
)

print(df_intervals.collect())


/tmp/ipykernel_3556/1804439988.py:6: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  df_intervals


shape: (184_186, 14)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ car ┆ _lo ┆ _up ┆ tim ┆ not ┆ not ┆ not ┆ notifi ┆ notif ┆ dista ┆ dista ┆ dista ┆ dista ┆ dista │
│ _id ┆ wer ┆ per ┆ est ┆ ifi ┆ ifi ┆ ifi ┆ cation ┆ icati ┆ nce_m ┆ nce_m ┆ nce_v ┆ nce_m ┆ nce_m │
│ --- ┆ _bo ┆ _bo ┆ amp ┆ cat ┆ cat ┆ cat ┆ _time_ ┆ on_ti ┆ ean   ┆ edian ┆ arian ┆ ax    ┆ in    │
│ str ┆ und ┆ und ┆ --- ┆ ion ┆ ion ┆ ion ┆ max    ┆ me_mi ┆ ---   ┆ ---   ┆ ce    ┆ ---   ┆ ---   │
│     ┆ ary ┆ ary ┆ dat ┆ _ti ┆ _ti ┆ _ti ┆ ---    ┆ n     ┆ f64   ┆ f64   ┆ ---   ┆ f64   ┆ f64   │
│     ┆ --- ┆ --- ┆ eti ┆ me_ ┆ me_ ┆ me_ ┆ durati ┆ ---   ┆       ┆       ┆ f64   ┆       ┆       │
│     ┆ dat ┆ dat ┆ me[ ┆ mea ┆ med ┆ var ┆ on[μs] ┆ durat ┆       ┆       ┆       ┆       ┆       │
│     ┆ eti ┆ eti ┆ μs] ┆ n   ┆ ian ┆ ian ┆        ┆ ion[μ ┆       ┆       ┆       ┆       ┆       │
│     ┆ me[ ┆ me[ ┆     ┆ --- ┆ --- ┆ ce  ┆        ┆ s]    ┆       ┆  